# Setup

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import json
import numpy as np
import cv2
from google.cloud import storage
import os
import time


# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: %s" % device)
# device = 'cpu'

device: cuda


In [2]:
with open('annotations/valid.json') as f:
    test_data = json.load(f)
with open('annotations/train.json') as f:
    train_data = json.load(f)
    
storage_client = storage.Client("pose_estimation_2")
bucket = storage_client.get_bucket('pose_estimation_2_dataset_mpii')

# NUM_TRAIN = 22246
NUM_TRAIN = 512
NUM_TEST = 2958

# Hyperparameters

In [3]:
batch_size = 4
epochs = 100
learning_rate = 0.0001

# Train Loop

In [4]:
import modules
import gc

from modules.unipose import UniPose
from modules.criterion.distribution_difference_loss import DistributionDifferenceLoss 
from modules.criterion.joint_max_mse_loss import JointMaxMSELoss

model = UniPose().to(device)
criterion = DistributionDifferenceLoss(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

alt_criterion = JointMaxMSELoss()

In [6]:
load_start = time.time()

# imagelist   = []
# kptlist     = []

torch_image = torch.zeros(NUM_TRAIN, 720, 960, 3)
kpt_tensor = torch.zeros(NUM_TRAIN, 16, 2)

# For each image, load the image
for i in range(NUM_TRAIN):
    img_name = train_data[i]['image']

    blob = bucket.blob('MPII/images/' +  img_name)
    blob.content_type = 'image/jpeg'
    image = np.asarray(bytearray(blob.download_as_string()))
    img = cv2.imdecode(image, cv2.IMREAD_UNCHANGED)

    kpt = np.asarray(train_data[i]['joints'], dtype=np.int32)

    if img.shape[0] != 960 or img.shape[1] != 720:
        kpt[:,0] = kpt[:,0] * (960/img.shape[1])
        kpt[:,1] = kpt[:,1] * (720/img.shape[0])
        img = cv2.resize(img,(960,720))
        img = np.array(img)

#     imagelist.append(img)
#     kptlist.append(kpt)
    torch_image[i,:,:,:] = torch.Tensor(img)
    kpt_tensor[i,:,:]   = torch.Tensor(kpt)
    
    if i % 1 == 0:
        print(f'Loaded {i+1} images')

image_load_time = time.time()

# construct image tensor and label tensor
# torch_image = torch.Tensor(imagelist)
torch_image = torch_image.permute(0, 3, 1, 2)
# kpt_tensor = torch.FloatTensor(kptlist)

tensor_build_time = time.time()

print(f'Image Load:{image_load_time - load_start}, Tensor Build:{tensor_build_time - image_load_time}')
print(f'Loaded {NUM_TRAIN} images, shape is {torch_image.shape}')

Loaded 1 images
Loaded 2 images
Loaded 3 images
Loaded 4 images
Loaded 5 images
Loaded 6 images
Loaded 7 images
Loaded 8 images
Loaded 9 images
Loaded 10 images
Loaded 11 images
Loaded 12 images
Loaded 13 images
Loaded 14 images
Loaded 15 images
Loaded 16 images
Loaded 17 images
Loaded 18 images
Loaded 19 images
Loaded 20 images
Loaded 21 images
Loaded 22 images
Loaded 23 images
Loaded 24 images
Loaded 25 images
Loaded 26 images
Loaded 27 images
Loaded 28 images
Loaded 29 images
Loaded 30 images
Loaded 31 images
Loaded 32 images
Loaded 33 images
Loaded 34 images
Loaded 35 images
Loaded 36 images
Loaded 37 images
Loaded 38 images
Loaded 39 images
Loaded 40 images
Loaded 41 images
Loaded 42 images
Loaded 43 images
Loaded 44 images
Loaded 45 images
Loaded 46 images
Loaded 47 images
Loaded 48 images
Loaded 49 images
Loaded 50 images
Loaded 51 images
Loaded 52 images
Loaded 53 images
Loaded 54 images
Loaded 55 images
Loaded 56 images
Loaded 57 images
Loaded 58 images
Loaded 59 images
Loaded

In [ ]:
epoch_losses = []

gc.collect()
torch.cuda.empty_cache()

# For each epoch
for epoch in range(epochs):
    epoch_loss = []

    # For each batch
    for start_i in range(0, NUM_TRAIN, batch_size):
        
        start_time = time.time()

        torch_image_batch = torch_image[ start_i: start_i + batch_size ,:,:,:].to(device)
        kpt_batch         = kpt_tensor[ start_i: start_i + batch_size ,:,:]

        tensor_transfer_time = time.time()
        
        # Train on batch
        optimizer.zero_grad()

        out = model(torch_image_batch)
        
        forward_pass_time = time.time()
        
        batch_loss = criterion(out, kpt_batch)
        epoch_loss.append(batch_loss.item())
        batch_loss.backward()

        optimizer.step()
        
        backprop_time = time.time()

        print(f'Epoch: {epoch}, Batch: {i // batch_size}, Batch Distribution Difference Loss: {batch_loss}, JointMaxMSELoss (to see if model is working): {alt_criterion(out, kpt_batch.to(device))}')
        print(f'Tensor Transfer:{tensor_transfer_time - start_time}, Forward:{forward_pass_time - tensor_transfer_time}, Backward:{backprop_time - forward_pass_time}, Total:{backprop_time - start_time}')
        
        gc.collect()
        torch.cuda.empty_cache()
        
    print(f'Epoch: {epoch}, Average Batch Loss: {sum(epoch_loss) / len(epoch_loss)}')
    epoch_losses.append(epoch_loss)

Epoch: 0, Batch: 127, Batch Distribution Difference Loss: 0.004100700840353966, JointMaxMSELoss (to see if model is working): 381951.34375
Tensor Transfer:0.00843191146850586, Forward:0.4327073097229004, Backward:20.81076955795288, Total:21.251908779144287
Epoch: 0, Batch: 127, Batch Distribution Difference Loss: 0.012345756404101849, JointMaxMSELoss (to see if model is working): 366475.78125
Tensor Transfer:0.02065134048461914, Forward:0.13741374015808105, Backward:20.82953119277954, Total:20.98759627342224
Epoch: 0, Batch: 127, Batch Distribution Difference Loss: 0.003395032836124301, JointMaxMSELoss (to see if model is working): 308833.28125
Tensor Transfer:0.020673751831054688, Forward:0.1411302089691162, Backward:20.830199718475342, Total:20.992003679275513
Epoch: 0, Batch: 127, Batch Distribution Difference Loss: 0.003656604792922735, JointMaxMSELoss (to see if model is working): 295416.9375
Tensor Transfer:0.020228862762451172, Forward:0.1370410919189453, Backward:20.81435394287